In [22]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd

In [23]:
df = pd.read_csv('https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/refs/heads/master/6_gradient_descent/insurance_data.csv')

In [24]:
X , y = df.drop(columns='bought_insurance') , df['bought_insurance']

In [25]:
X.columns

Index(['age', 'affordibility'], dtype='object')

In [26]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)

In [27]:
from sklearn.preprocessing import StandardScaler
st = StandardScaler()
for col in X_train.columns:
    X_train[col] = st.fit_transform(X_train[[col]])
    X_test[col] = st.transform(X_test[[col]])

In [28]:
model = keras.Sequential([
    keras.layers.Dense(100,input_shape=(2,),activation='tanh',kernel_initializer='ones',bias_initializer='zeros'),
    keras.layers.Dense(1,activation='sigmoid')
])

In [29]:
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = ['accuracy']
)

In [30]:
# model.fit(X_train,y_train,epochs=100)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 1.0841 - accuracy: 0.1364
Epoch 2/100
1/1 [==============================] - 0s 36ms/step - loss: 1.0384 - accuracy: 0.1818
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9940 - accuracy: 0.1818
Epoch 4/100
1/1 [==============================] - 0s 31ms/step - loss: 0.9512 - accuracy: 0.1818
Epoch 5/100
1/1 [==============================] - 0s 32ms/step - loss: 0.9098 - accuracy: 0.1818
Epoch 6/100
1/1 [==============================] - 0s 24ms/step - loss: 0.8701 - accuracy: 0.1818
Epoch 7/100
1/1 [==============================] - 0s 19ms/step - loss: 0.8321 - accuracy: 0.1818
Epoch 8/100
1/1 [==============================] - 0s 33ms/step - loss: 0.7957 - accuracy: 0.2727
Epoch 9/100
1/1 [==============================] - 0s 45ms/step - loss: 0.7611 - accuracy: 0.3636
Epoch 10/100
1/1 [==============================] - 0s 32ms/step - loss: 0.7282 - accuracy: 0.3636
Epoch 11/100
1/1 [===

KeyboardInterrupt: 

In [ ]:
y_pred = model.predict(X_test)

1/1 [==============================] - 6s 6s/step


In [ ]:
y_pred = [1 if i > 0.5 else 0 for i in y_pred]

In [ ]:
acc = sum(y_pred==y_test) / len(y_test)

In [ ]:
acc*100

0.0

In [ ]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

In [ ]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [ ]:
class my_nn:
    def __init__(self):
        self.w1 = 0
        self.w2 = 0
        self.b = 0

    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.b = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold)

    def predict (self , X_test):
        y_pred = self.w1 * X_test['age'] + self.w2 * X_test['affordibility'] + self.b
        y_pred = sigmoid(y_pred) 
        y_pred = [1 if i > 0.5 else 0 for i in y_pred]
        return y_pred

    def gradient_descent(self , age , affordibility , y_true , epoch , loss_threshold):

        w1 , w2 , b , lr , sample_size = 1 , 1 , 0 , 0.01 , len(age)

        for _ in range(epoch):
            
            
            linear = w1 * age + w2 * affordibility + b

            y_pred = sigmoid(linear)
            loss = log_loss(y_true,y_pred)

            w1d = (1/sample_size)*np.dot(np.transpose(age),(y_pred-y_true)) 
            w2d = (1/sample_size)*np.dot(np.transpose(affordibility),(y_pred-y_true)) 

            bias_d = np.mean(y_pred-y_true)
            w1 = w1 - lr * w1d
            w2 = w2 - lr * w2d
            b = b - lr * bias_d
            
            if loss <= loss_threshold:
                break

        return w1,w2,b

In [ ]:
model = my_nn()

In [ ]:
model.fit(X_train,y_train,-16,0.1)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred

[1, 0, 0, 1, 0, 1]

In [ ]:
y_test

25    1
12    0
6     0
15    1
20    0
5     1
Name: bought_insurance, dtype: int64

In [ ]:
def gradient_descent(age , affordibility , y_true , epoch , loss_threshold):
        w1 , w2 , b , lr , sample_size = 1 , 1 , 0 , 0.01 , len(age)

        for _ in range(epoch):
            
            linear = w1 * age + w2 * affordibility + b
            
            y_pred = sigmoid(linear)
            
            loss = log_loss(y_true,y_pred)

            w1d = (1/sample_size)*np.dot(np.transpose(age),(y_pred-y_true)) 
            w2d = (1/sample_size)*np.dot(np.transpose(affordibility),(y_pred-y_true)) 

            bias_d = np.mean(y_pred-y_true)
            
            w1 = w1 - lr * w1d
            w2 = w2 - lr * w2d
            b = b - lr * bias_d

            if loss <= loss_threshold:
                break
            
        return w1,w2,b